In [1]:
import face_alignment
from skimage import io
import numpy as np
import pickle
from mpl_toolkits.mplot3d import Axes3D
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import os
from pathlib import Path
import cv2
import tqdm
%matplotlib inline

In [2]:
source_dir = Path("data/source/")
source_dir.mkdir(exist_ok=True)

save_dir = source_dir.joinpath("images")
save_dir.mkdir(exist_ok=True)

label_dir = source_dir.joinpath("label")
label_dir.mkdir(exist_ok=True)

label_on_image_dir = source_dir.joinpath("label_on_image")
label_on_image_dir.mkdir(exist_ok=True)

In [ ]:
voxceleb_dir = source_dir.joinpath("mp4")
voxceleb_list = os.listdir(voxceleb_dir)

input_size = (224, 224)

for celeb in voxceleb_list:
    each_celeb_dir = voxceleb_dir.joinpath(celeb)
    each_celeb_save_dir = save_dir.joinpath(celeb)
    each_celeb_save_dir.mkdir(exist_ok=True)
    
    pbar = tqdm.tqdm(os.listdir(each_celeb_dir))    
    for situation in pbar:
        pbar.set_description("%s" % celeb)
        each_celeb_each_dir = each_celeb_dir.joinpath(situation)
        each_celeb_each_save_dir = each_celeb_save_dir.joinpath(situation)
        each_celeb_each_save_dir.mkdir(exist_ok=True)
        
        for vid in os.listdir(each_celeb_each_dir):
            vid_name, _ = vid.split(".")
            each_celeb_cap_dir = each_celeb_each_save_dir.joinpath(vid_name)
            each_celeb_cap_dir.mkdir(exist_ok=True)
            cap = cv2.VideoCapture(str(each_celeb_each_dir.joinpath(vid)))
            
            if len(os.listdir(str(each_celeb_cap_dir))) < 5: # 프레임 단위로 캡처하지 않았다면, 캡처 실행
                count = 0
                
                while cap.isOpened():
                    flag, frame = cap.read()
                    if flag == False:
                        break
                    shape_dst = np.min(frame.shape[:2])
                    oh = (frame.shape[0] - shape_dst) // 2
                    ow = (frame.shape[1] - shape_dst) // 2
                    
                    img = frame[oh:oh+shape_dst, ow:ow+shape_dst]
                    img = cv2.resize(img, input_size)
                    img_name = "{:05}.jpg".format(count)
                    
                    cv2.imwrite(str(each_celeb_cap_dir.joinpath(img_name)), img)
                    
                    count += 1

id00817:  19%|█████████████▌                                                           | 10/54 [01:26<04:37,  6.31s/it]

In [63]:
preds = {}
for celeb in voxceleb_list:
    each_celeb_save_dir = save_dir.joinpath(celeb)
    
    pbar = tqdm.tqdm(os.listdir(each_celeb_save_dir))    
    for situation in pbar:
        pbar.set_description("%s" % celeb)        
        each_celeb_each_save_dir = each_celeb_save_dir.joinpath(situation)
        for vid in os.listdir(each_celeb_each_save_dir):            
            if ".pkl" in vid:
                continue
                            
            if len(os.listdir(str(label_dir.joinpath(celeb+"/"+situation+"/"+vid))))<5: # 라벨을 만들지 않았다면, preds 실행
                each_celeb_cap_dir = each_celeb_each_save_dir.joinpath(vid)
                pickle_name = vid+".pkl"

                if pickle_name in os.listdir(str(each_celeb_each_save_dir)):                
                    preds[celeb+"_"+situation+"_"+pickle_name] = pickle.load(open(str(each_celeb_each_save_dir.joinpath(pickle_name)), "rb"))

                else:
                    fa = face_alignment.FaceAlignment(face_alignment.LandmarksType._3D, flip_input=False)
                    preds[celeb+"_"+situation+"_"+vid] = fa.get_landmarks_from_directory(str(each_celeb_cap_dir), show_progress_bar=False)
                    pickle.dump(preds[celeb+"_"+situation+"_"+vid], open(str(each_celeb_each_save_dir.joinpath(pickle_name)), "wb"), pickle.HIGHEST_PROTOCOL)





























  0%|                                                                                           | 0/32 [00:00<?, ?it/s]



























id00017:   0%|                                                                                  | 0/32 [00:00<?, ?it/s]



























id00017:   3%|██▎                                                                       | 1/32 [00:42<22:06, 42.78s/it]



























id00017:   3%|██▎                                                                       | 1/32 [00:42<22:06, 42.78s/it]

id00017_01dfn2spqyE_00001.pkl


In [82]:
colors = {'white' : (255, 255, 255),
         'red' : (255, 0, 0),
         'green' : (0, 128, 0),
         'blue' : (0, 0, 255),
         'yellow' : (255, 255, 0),
         'cyan' : (0, 255, 255),
         'magenta' : (255, 0, 255),
         'orange' : (255, 165, 0),
         'purple' : (128, 0, 128)}

pbar = tqdm.tqdm(preds.items())
for pred_key, pred_val in pbar:
    celeb, situation, vid = pred_key.split("_")    
    
    pbar.set_description("%s" % celeb) 
    
    c_dir = save_dir.joinpath(celeb)
    c_s_dir = c_dir.joinpath(situation)
    c_s_v_dir = c_s_dir.joinpath(vid)
    
    c_label_dir = label_dir.joinpath(celeb)
    c_label_dir.mkdir(exist_ok=True)
    c_s_label_dir = c_label_dir.joinpath(situation)
    c_s_label_dir.mkdir(exist_ok=True)
    c_s_v_label_dir = c_s_label_dir.joinpath(vid)
    c_s_v_label_dir.mkdir(exist_ok=True)
    
    c_label_on_dir = label_on_image_dir.joinpath(celeb)
    c_label_on_dir.mkdir(exist_ok=True)
    c_s_label_on_dir = c_label_on_dir.joinpath(situation)
    c_s_label_on_dir.mkdir(exist_ok=True)
    c_s_v_label_on_dir = c_s_label_on_dir.joinpath(vid)
    c_s_v_label_on_dir.mkdir(exist_ok=True)
    
    for k, v in pred_val.items():        
        lm = {'1' : (np.array(v[0][0:17,0:2], np.int32), colors['green']), # 턱
              '2' : (np.array(v[0][17:22,0:2], np.int32), colors['orange']), # 왼쪽 눈썹
              '3' : (np.array(v[0][22:27,0:2], np.int32), colors['orange']), # 오른쪽 눈썹
              '4' : (np.array(v[0][27:31,0:2], np.int32), colors['blue']), # 콧대
              '5' : (np.array(v[0][31:36,0:2], np.int32), colors['blue']), # 콧망울
              '6' : (np.vstack((np.array(v[0][36:42,0:2], np.int32), np.array(v[0][36,0:2], np.int32))), colors['red']), # 왼쪽 눈
              '7' : (np.vstack((np.array(v[0][42:48,0:2], np.int32), np.array(v[0][42,0:2], np.int32))), colors['red']), # 오른쪽 눈
              '8' : (np.vstack((np.array(v[0][48:60,0:2], np.int32), np.array(v[0][48,0:2], np.int32))), colors['purple']), # 바깥쪽 입술
              '9' : (np.vstack((np.array(v[0][60:68,0:2], np.int32), np.array(v[0][60,0:2], np.int32))), colors['purple'])} # 안쪽 입술
        
        img = cv2.imread(str(c_s_v_dir.joinpath("{:05}.jpg".format(count))))
        label = np.zeros((input_size[0], input_size[1], 3), np.uint8)
        
        for mark, color in lm.values():
            cv2.polylines(img=label, pts=[mark], isClosed=False, color=color[::-1], thickness=2)
            cv2.polylines(img=img, pts=[mark], isClosed=False, color=color[::-1], thickness=2)
            
        cv2.imwrite(str(c_s_v_label_dir.joinpath(k.split("\\")[-1])),label)
        cv2.imwrite(str(c_s_v_label_on_dir.joinpath(k.split("\\")[-1])),img)






























  0%|                                                                                            | 0/2 [00:00<?, ?it/s]




























id00017:   0%|                                                                                   | 0/2 [00:00<?, ?it/s]




























id00017:  50%|█████████████████████████████████████▌                                     | 1/2 [00:01<00:01,  1.36s/it]




























id00017:  50%|█████████████████████████████████████▌                                     | 1/2 [00:01<00:01,  1.36s/it]




























id00017: 100%|███████████████████████████████████████████████████████████████████████████| 2/2 [00:03<00:00,  1.71s/it]